In [ ]:
import pandas as pd
import texthero as hero
import matplotlib.pyplot as plt
import seaborn as sns
from texthero import preprocessing
import nltk
from nltk import FreqDist
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity



In [ ]:
# Chargement du jeu de données
data= pd.read_csv("../data/Emotion_final.csv")                                                                                                                                                                                                          

In [ ]:
data

In [ ]:
data.info()

In [ ]:
data["Emotion"].unique()

In [ ]:
data.isna().sum()

In [ ]:
# Répartition des textes par émotions
emotion_counts = data['Emotion'].value_counts()
emotion_counts

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(x=emotion_counts.index, y=emotion_counts.values)
plt.xlabel('Emotions')
plt.ylabel('Nombre de textes')
plt.title('Répartition des textes par émotions')
plt.show()

In [ ]:
# Prétraitement des données
custom_pipeline = [
    preprocessing.fillna,
    preprocessing.lowercase,
    preprocessing.remove_digits,
    preprocessing.remove_punctuation,
    preprocessing.remove_diacritics,
    preprocessing.remove_whitespace,
    preprocessing.remove_stopwords
]

data['clean_text'] = hero.clean(data['Text'], pipeline=custom_pipeline)


In [ ]:
data

In [ ]:
# Extraction des mots les plus courants pour chaque sentiment
common_words_per_emotion = {}
for emotion in data['Emotion'].unique():
    mask = data['Emotion'] == emotion
    text = data[mask]['clean_text']
    words = ' '.join(text).split()
    word_freq = FreqDist(words)
    top_words = [word for word, freq in word_freq.most_common(30) if word not in stopwords.words('english')]
    common_words_per_emotion[emotion] = top_words


In [ ]:
# Affichage des 30 mots les plus courants pour chaque sentiment
for emotion, words in common_words_per_emotion.items():
    print(f"Emotion: {emotion}")
    print(words)
    print()

In [ ]:
# # Création d'une matrice de proximité
# emotions = list(common_words_per_emotion.keys())
# distance_matrix = pd.DataFrame(index=emotions, columns=emotions)

# for i, emotion1 in enumerate(emotions):
#     for j, emotion2 in enumerate(emotions):
#         if i == j:
#             distance_matrix.loc[emotion1, emotion2] = 1.0
#         else:
#             common_words1 = common_words_per_emotion[emotion1]
#             common_words2 = common_words_per_emotion[emotion2]
#             vectorizer = CountVectorizer(vocabulary=set(common_words1 + common_words2))
#             vectors = vectorizer.fit_transform([common_words1, common_words2])
#             similarity = cosine_similarity(vectors)[0][1]
#             distance_matrix.loc[emotion1, emotion2] = similarity